# Analysis of the astrocytes RNA-seq data from HPT, CTX, and Hippo  

## Table of contents:

* <a href=#Load>Load packages and set global variables</a>
* <a href=#Dataloading>Loading data, preprocessing and make DEseq object</a>
* <a href=#Visualisataion>Visualisation of the data</a>
* <a href=#DGE>Differential gene expression - diet effect</a>
* <a href=#Celltype>Heatmap marker genes</a>
* <a href=#HeatmapDGE>Heatmap DGE</a>
* <a href=#GOenrichment>GO pathway enrichment</a>

## Load packages and set global variables

In [1]:
inst = suppressMessages(lapply(c('DESeq2',
                                 'vidger',
                                 'pheatmap',
                                 'ggplot2',
                                 'EnhancedVolcano',
                                 'VennDiagram',
                                 'org.Mm.eg.db',
                                 'clusterProfiler',
                                 'UpSetR',
                                 'readxl',
                                 'tidyverse',
                                 'KEGGREST'),
                               library,
                               character.only=TRUE)
) 

Warning message:
“package ‘readxl’ was built under R version 4.0.5”
Warning message:
“package ‘tidyr’ was built under R version 4.0.5”
Warning message:
“package ‘readr’ was built under R version 4.0.5”
Warning message:
“package ‘dplyr’ was built under R version 4.0.5”


In [2]:
setwd("/Users/viktorian.miok/Documents/consultation/Ismael/RNAseq_Agrp")
options(repr.plot.width=8, repr.plot.height=8)
my_palette = colorRampPalette(c("blue", "white", "red"))(n=255)

Set whether anndata objects are recomputed or loaded.

In [3]:
bool_recomp = TRUE

Set whether to produce plots, set to False for test runs.

In [4]:
bool_plot = TRUE

## Loading data, preprocessing and make DEseq object 

In [5]:
dat <- read.csv(file="./data/data_raw.csv", 
                header=TRUE,
                row.names=1,
                sep=",")[,-1]

In [6]:
if(bool_recomp){  
    # make a meta data
    id = c("21L011015","21L011016","21L011017","21L011018","21L011019","21L011020","21L011021",
           "21L011022","21L011023","21L011024","21L011025","21L011026") # "21L011014",

    condition = c("DREADD","DREADD","DREADD","HFHS DIET","HFHS DIET","HFHS DIET","HFHS DIET","DREADD",
                  "SC DIET","SC DIET","SC DIET","SC DIET")

    metaData=data.frame(id, condition)

    keep <- rowSums(dat >= 10) >= 5
    dat <- dat[keep,]

    #make a deseq2 object
    dds <- DESeqDataSetFromMatrix(countData=dat,
                                  colData=metaData, 
                                  design =~ condition
    )
    dds <- DESeq(dds)
    save(dds, file="data/dds.RData")
} else {
    load("data/dds.RData")
}

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
 

## Visualisation of the data

In [7]:
# variance stabilizing transformation data
vsd = assay(vst(dds, blind=TRUE))

In [8]:
pdf(file="results/PCA.pdf", width=10, height=10)
    pcaData <- plotPCA(vst(dds),
                       intgroup=c("condition"),
                       returnData=TRUE
    )
    percentVar = round(100 * attr(pcaData, "percentVar"))

    ggplot(pcaData, aes(x=PC1, y=PC2, color=condition)) +
           geom_point(aes(color=condition), size=5) + 
           geom_point(shape=1, color="black", size=5) + 
           xlab(paste0("PC1: ",percentVar[1], "% variance")) + 
           ylab(paste0("PC2: ",percentVar[2], "% variance")) +
           theme(axis.text=element_text(size=20), 
                 axis.title=element_text(size=20), 
                 legend.title=element_text(size=20),
                 legend.text=element_text(size=20),
                 plot.title=element_text(size=20)) + 
           scale_color_manual(values=c("chartreuse", "cornflowerblue", "red"))
dev.off()

pdf 
  2

In [9]:
if(bool_plot){  
    pheatmap(vsd,
             cluster_rows=TRUE, 
             cluster_cols=TRUE, 
             scale="row", 
             show_rownames=FALSE, 
             color=my_palette,
             filename="results/heatmap_all_data.pdf",
             annotation_col=as.data.frame(colData(dds)[,c("id", "condition")]),
             fontsize=10,
             width=10,
             height=10,
             breaks=seq(-3, 3, length.out=255)
    )
}

## Differential gene expression - diet effect

In [10]:
if(bool_recomp){ 
    dread_sc <- results(dds,
                        contrast=c("condition", "DREADD", "SC DIET")
    )
    hfhs_sc <- results(dds, 
                       contrast=c("condition", "HFHS DIET", "SC DIET")
    )
    hfhs_dread <- results(dds, 
                          contrast=c("condition", "HFHS DIET", "DREADD")
    )

    dge <- list(dread_sc=as.data.frame(dread_sc[complete.cases(dread_sc), ]),
                hfhs_sc=as.data.frame(hfhs_sc[complete.cases(hfhs_sc), ]),
                hfhs_dread=as.data.frame(hfhs_dread[complete.cases(hfhs_dread), ])
    )
    save(dge, file="data/dge.RData")
} else {
    load("data/dge.RData")
}

In [11]:
pdf(file="results/volcano_DREADD_vs_SC.pdf", width=10, height=10)
    EnhancedVolcano(dge$dread_sc,
                    lab=rownames(dge$dread_sc),
                    pCutoff=0.05,
                    FCcutoff=100,
                    axisLabSize=30,
                    x='log2FoldChange',
                    y='padj',
                    title=NULL,
                    subtitle=NULL,
                    caption=NULL,
                    ylim=c(0, 3),
                    xlim=c(-3, 3),
                    col=c("grey30", "grey30", "red2", "red2"),
                    legendLabels=c('NS', expression(Log[2]~FC), 'p-value>0.05', expression(p-value~and~log[2]~FC)),
                    legendLabSize=20
    )
dev.off()

Warning message:
“Removed 2 rows containing missing values (geom_vline).”


pdf 
  2

In [12]:
pdf(file="results/volcano_HFHS_vs_SC.pdf", width=10, height=10)
    EnhancedVolcano(dge$hfhs_sc,
                    lab=rownames(dge$hfhs_sc),
                    pCutoff=0.05,
                    FCcutoff=100,
                    axisLabSize=30,
                    x='log2FoldChange',
                    y='padj',
                    title=NULL,
                    subtitle=NULL,
                    caption=NULL,
                    ylim=c(0, 3),
                    xlim=c(-3, 3),
                    col=c("grey30", "grey30", "red2", "red2"),
                    legendLabels=c('NS', expression(Log[2]~FC), 'p-value>0.05', expression(p-value~and~log[2]~FC)),
                    legendLabSize=20
    )
dev.off()

Warning message:
“Removed 2 rows containing missing values (geom_vline).”


pdf 
  2

In [13]:
pdf(file="results/volcano_HFHS_vs_DREADD.pdf", width=10, height=10)
    EnhancedVolcano(dge$hfhs_dread,
                    lab=rownames(dge$hfhs_dread),
                    pCutoff=0.05,
                    FCcutoff=100,
                    axisLabSize=30,
                    x='log2FoldChange',
                    y='padj',
                    title=NULL,
                    subtitle=NULL,
                    caption=NULL,
                    ylim=c(0, 3),
                    xlim=c(-3, 3),
                    col=c("grey30", "grey30", "red2", "red2"),
                    legendLabels=c('NS', expression(Log[2]~FC), 'p-value>0.05', expression(p-value~and~log[2]~FC)),
                    legendLabSize=20
    )
dev.off()

Warning message:
“Removed 2 rows containing missing values (geom_vline).”


pdf 
  2

In [14]:
pdf(file="results/Nr_sig_DGE", width=10, height=10)
    df = data.frame(Tissue=c("DREADD_vs_SC", "HFHS_vs_SC", "HFHS_vs_DREADD"),
                    Nr_significant_genes=c(sum(dge$dread_sc[,6] < 0.05, na.rm=TRUE),
                                           sum(dge$hfhs_sc[,6] < 0.05, na.rm=TRUE),
                                           sum(dge$hfhs_dread[,6] < 0.05, na.rm=TRUE))
    )
    ggplot(df, aes(x=Tissue, y=Nr_significant_genes, fill=Tissue))  +
           geom_bar(stat="identity") + 
            theme(axis.text=element_text(size=20), 
                  axis.title=element_text(size=20), 
                  legend.title=element_text(siz=20),
                  legend.text=element_text(size=20),
                  plot.title=element_text(size=20)) +
           theme(legend.position="none") +
           ggtitle("Number significant genes (p<0.05)") +
           scale_fill_manual(values=c("lightblue", "tan1", "lightgreen"))
dev.off()

pdf 
  2

In [15]:
pdf(file="results/venn_DEG.pdf", width=8, height=8)
    vp = venn.diagram(list('DREADD_vs_SC'=rownames(dge$dread_sc[dge$dread_sc[,6] < 0.05,]), 
                           'HFHS_vs_SC'=rownames(dge$hfhs_sc[dge$hfhs_sc[,6] < 0.05,]),
                           'HFHS_vs_DREADD'=rownames(dge$hfhs_dread[dge$hfhs_dread[,6] < 0.05,])),
                     fill=c("orange", "mediumpurple1", "lightgreen"),
                     filename=NULL,
                     cex=3.5, 
                     #col="transparent",
                     cat.cex=1.1,
                     margin=0.01,
                     main.cex=3,
                     main=NULL);
    grid.draw(vp)  
    #overlap <- calculate.overlap(venn.diagram(list('DREADD_vs_SC'=rownames(dge$dread_sc[dge$dread_sc[,6]<0.05,]), 
    #                                               'HFHS_vs_SC'=rownames(dge$hfhs_sc[dge$hfhs_sc[,6]<0.05,]),
    #                                               'HFHS_vs_DREADD'=rownames(dge$hfhs_dread[dge$hfhs_dread[,6]<0.05,])))
dev.off()

pdf 
  2

## Heatmap DGE

#### DREADD vs. SC

In [16]:
if(bool_plot){  
    vsd_dread_sc = vsd[rownames(vsd)%in%rownames(dge$dread_sc[dge$dread_sc[,6] < 0.05,]),] 
    pheatmap(vsd_dread_sc[,c(1:3,9:12)], 
             scale="row",
             show_rownames=FALSE,
             show_colnames=TRUE,
             color=my_palette, 
             cluster_cols=FALSE,
             legend=FALSE,
             fontsize=15,
             filename="results/heatmap_DREADD-SC.pdf",
             width=5,
             height=8,
             breaks=seq(-2, 2, length.out=255)
    ) 
    write.csv(dge$dread_sc[dge$dread_sc[,6] < 0.05,], 
              file="tables/DEG_dread_sc.csv",
              row.names=TRUE
    )
}

#### HFHS vs. SC

In [17]:
if(bool_plot){  
    vsd_hfhs_sc = vsd[rownames(vsd)%in%rownames(dge$hfhs_sc[dge$hfhs_sc[,6] < 0.05,]),] 
    pheatmap(vsd_hfhs_sc[,c(4:7,9:12)], 
             scale="row",
             show_rownames=FALSE,
             show_colnames=TRUE,
             color=my_palette, 
             cluster_cols=FALSE,
             legend=FALSE,
             fontsize=15,
             filename="results/heatmap_HFHS-SC.pdf",
             width=5,
             height=8,
             breaks=seq(-2, 2, length.out=255)
    ) 
    write.csv(dge$hfhs_sc[dge$hfhs_sc[,6] < 0.05,], 
              file="tables/DEG_hfhs_sc.csv",
              row.names=TRUE
    )
}

#### HFHS vs. DREADD

In [18]:
if(bool_plot){  
    vsd_hfhs_dread = vsd[rownames(vsd)%in%rownames(dge$hfhs_dread[dge$hfhs_dread[,6] < 0.05,]),] 
    pheatmap(vsd_hfhs_dread[,c(1:3,8,4:7)], 
             scale="row",
             show_rownames=FALSE,
             show_colnames=TRUE,
             color=my_palette, 
             cluster_cols=FALSE,
             legend=FALSE,
             fontsize=15,
             filename="results/heatmap_HFHS-DREADD.pdf",
             width=5,
             height=8,
             breaks=seq(-2, 2, length.out=255)
    ) 
    write.csv(dge$hfhs_dread[dge$hfhs_dread[,6] < 0.05,], 
              file="tables/DEG_hfhs_dread.csv",
              row.names=TRUE
    )
}

## Plot the genes

In [19]:
genes = c('Agrp','Npy','Pomc','Oxt','Avp','Gfap','Iba1','Vim', #general
          'Aldh1l1','Aqp4','Atp1b2','Gfap','Gja1','Gjb6','S100b','Slc1a2','Slc1a3',  # astrocytes
          'Snap25','Syp','Syt1','Tubb3',  #neurons
          'Aif1','Itgam',   # microglia
          'Mag','Mog',   # oligodendrocytes
          'Slco1c1',   # endothelial cells
          'Des','Mustn1','Pdgfrb')  #mural cells)

In [20]:
if(bool_plot){  
    vsd_gen = vsd[rownames(vsd)%in%c('Agrp','Npy','Pomc','Oxt','Avp','Gfap','Iba1','Vim'),] 
    pheatmap(vsd_gen[,c(1:3,8,4:7,9:12)], 
             scale="row",
             show_rownames=TRUE,
             show_colnames=TRUE,
             color=my_palette, 
             cluster_cols=FALSE,
             cluster_rows=FALSE,
             legend=FALSE,
             fontsize=15,
             filename="results/heatmap_general-markers.pdf",
             width=5,
             height=8,
             breaks=seq(-2, 2, length.out=255)
    ) 
}

In [21]:
if(bool_plot){  
    vsd_gen = vsd[rownames(vsd)%in%c('Aldh1l1', 'Aqp4', 'Atp1b2', 'Gfap', 'Gja1', 'Gjb6','S100b','Slc1a2','Slc1a3'),] 
    pheatmap(vsd_gen[,c(1:3,8,4:7,9:12)], 
             scale="row",
             show_rownames=TRUE,
             show_colnames=TRUE,
             color=my_palette, 
             cluster_cols=F,
             cluster_rows=F,
             legend=FALSE,
             fontsize=15,
             filename="results/heatmap_astocyte-markers.pdf",
             width=5,
             height=8,
             breaks=seq(-2, 2, length.out=255)
    ) 
}

In [22]:
if(bool_plot){  
    vsd_gen = vsd[rownames(vsd)%in%c('Snap25', 'Syp', 'Syt1', 'Tubb3'),] 
    pheatmap(vsd_gen[,c(1:3,8,4:7,9:12)], 
             scale="row",
             show_rownames=TRUE,
             show_colnames=TRUE,
             color=my_palette, 
             cluster_cols=F,
             cluster_rows=F,
             legend=FALSE,
             fontsize=15,
             filename="results/heatmap_neuro-markers.pdf",
             width=5,
             height=8,
             breaks=seq(-2, 2, length.out=255)
    ) 
}

In [23]:
if(bool_plot){  
    vsd_gen = vsd[rownames(vsd)%in%c('Aif1', 'Itgam', 'Mag', 'Mog'),] 
    pheatmap(vsd_gen[,c(1:3,8,4:7,9:12)], 
             scale="row",
             show_rownames=TRUE,
             show_colnames=TRUE,
             color=my_palette, 
             cluster_cols=FALSE,
             cluster_rows=FALSE,
             legend=FALSE,
             fontsize=15,
             filename="results/heatmap_micro&oligo-markers.pdf",
             width=5,
             height=8,
             breaks=seq(-2, 2, length.out=255)
    ) 
}

In [24]:
if(bool_plot){  
    vsd_gen = vsd[rownames(vsd)%in%c('Slco1c1', 'Des', 'Mustn1', 'Pdgfrb'),] 
    pheatmap(vsd_gen[,c(1:3,8,4:7,9:12)], 
             scale="row",
             show_rownames=TRUE,
             show_colnames=TRUE,
             color=my_palette, 
             cluster_cols=FALSE,
             cluster_rows=FALSE,
             legend=FALSE,
             fontsize=15,
             filename="results/heatmap_endo&mural-markers.pdf",
             width=5,
             height=8,
             breaks=seq(-2, 2, length.out=255)
    ) 
}

## GO - enrichment

#### DREADD vs. SC

In [25]:
sg.dread_sc <- bitr(rownames(dge$dread_sc[dge$dread_sc[,6] < 0.05,]), 
                    fromType="SYMBOL",
                    toType="ENTREZID", 
                    OrgDb=org.Mm.eg.db
)
go.dread_sc <- enrichGO(sg.dread_sc[,2],
                        'org.Mm.eg.db', 
                        ont="BP", 
                        pvalueCutoff=0.1
)

p <- dotplot(go.dread_sc, 
             showCategory=30
)
ggplot2::ggsave(p, 
                filename="results/GO_BP_dread_sc.pdf", 
                width=10, 
                height=10
)
write.csv(data.frame(go.dread_sc), 
          file="tables/GO_BP_dread_sc.csv",
          row.names=FALSE
)

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(rownames(dge$dread_sc[dge$dread_sc[, 6] < 0.05, ]), fromType = "SYMBOL", :
“4.2% of input gene IDs are fail to map...”


#### HFHS vs. SC

In [26]:
sg.hfhs_sc <- bitr(rownames(dge$hfhs_sc[dge$hfhs_sc[,6] < 0.05,]), 
                    fromType="SYMBOL",
                    toType="ENTREZID", 
                    OrgDb=org.Mm.eg.db
)
go.hfhs_sc <- enrichGO(sg.hfhs_sc[,2],
                       'org.Mm.eg.db', 
                       ont="BP", 
                       pvalueCutoff=0.1
)
p <- dotplot(go.hfhs_sc,
             showCategory=30
)
ggplot2::ggsave(p,
                filename="results/GO_BP_hfhs_sc.pdf",
                width=10, 
                height=10
)
write.csv(data.frame(go.hfhs_sc),
                     file="tables/GO_BP_hfhs_sc.csv",
                     row.names=FALSE
)

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(rownames(dge$hfhs_sc[dge$hfhs_sc[, 6] < 0.05, ]), fromType = "SYMBOL", :
“4.14% of input gene IDs are fail to map...”
